# CFT – Saliva Analysis

## Setup and Helper Functions

In [ ]:
import re
from pathlib import Path

import pandas as pd
import numpy as np
import pingouin as pg

import matplotlib.pyplot as plt
import seaborn as sns

import biopsykit as bp
from biopsykit.protocols import MIST

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
plt.close("all")

palette = bp.colors.fau_palette
sns.set_theme(context="notebook", style="ticks", palette=palette)

plt.rcParams['figure.figsize'] = (10, 5)
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['mathtext.default'] = "regular"

palette

## Import Data

In [ ]:
base_path = Path("../../../")
data_path = base_path.joinpath("data/for_analysis")

results_path = base_path.joinpath("results")
plot_path = results_path.joinpath("plots")
stats_path = results_path.joinpath("statistics")

bp.utils.file_handling.mkdirs([results_path, plot_path, stats_path])

In [ ]:
cort_samples = bp.io.load_long_format_csv(
    data_path.joinpath("cortisol_samples.csv")
)

cort_features = bp.io.load_long_format_csv(
    data_path.joinpath("cortisol_features.csv")
)

display(cort_samples.head())
display(cort_features.head())

Drop cortisol sample `S0` for further analysis (only used to check for high cortisol baseline)

In [ ]:
cort_samples = cort_samples.drop("S0", level="sample")

In [ ]:
sample_times = [-1, 0, 10, 20, 30, 40] # times without S0

hue_order=["Control", "CFT"]

In [ ]:
mist = MIST.from_file(data_path.joinpath("../mist_cft.json"))
mist

In [ ]:
mist.add_saliva_data(cort_samples, "cortisol", sample_times=sample_times)

## Descriptive Analysis

Mean cortisol increase after MIST:

In [ ]:
cort_max_inc = cort_features.xs("max_inc_percent", level="saliva_feature")

cort_max_inc.groupby("condition").mean()

## Statistical Analysis

### MIST-induced Cortisol Increase

**Population**: Control group

**Analysis**: Check whether cortisol increased significantly after the MIST
* Procedure: Paired t-tests between cortisol sample before MIST (`S1`) and the maximum of all samples after MIST (`S2`-`S6`)
* Expected Result: Significant cortisol increase as response fo the MIST

**Findings**: 
* The MIST causes an effective activation of the HPA axis, indicated by a significant increase in cortisol after the MIST for the Control group

#### Prepare Data

In [ ]:
cort_s1 = cort_samples.xs("S1", level="sample")
cort_max = cort_samples.drop(["S0", "S1"], level="sample").groupby(["condition", "subject"]).max()
cort_analysis = pd.concat({"Pre": cort_s1, "Post": cort_max}, names=["time"])
cort_analysis = cort_analysis.reorder_levels(["condition", "subject", "time"]).sort_index()

cort_analysis.head()

#### Statistics

In [ ]:
steps = [
    ("prep", "normality"),
    ("test", "pairwise_ttests")
]
params = {
    "dv": "cortisol",
    "within": "time",
    "subject": "subject",
    "groupby": "condition"
}

stats = bp.stats.StatsPipeline(steps, params)
stats.apply(cort_analysis)

stats.export_statistics(stats_path.joinpath("stats_cortisol_response_mist.xlsx"))
stats.display_results()

### Effect of CFT on Acute Stress Response

**Population**: Control vs. CFT group

**Analysis**: 
1. *Interaction Condition x Time*: Check whether the CFT intervention interacts with HPA axis activity
    * Procedure:
        * Mixed-ANOVA to determine interaction effect between *Condition* and *Time*
        * In case of significant interaction effect: Post-hoc test to assess during which at sampling times cortisol samples were significantly different
    * Expected Result: Significant interaction effect between *Condition* and *Time*, cortisol levels start to significantly differ after the MIST

2. *Cortisol Features*: Check whether the CFT intervention causes an inhibition of the HPA axis, leading to less cortisol secretion
    * Procedure (for each cortisol feature): 
        * t-test between Control and CFT group
    * Expected Result: Significant differences between Control and CFT group: lower AUC, lower slope, lower maximum cortisl increase


**Findings**: 
* tbd

#### Interaction *Condition* x *Time*

##### Prepare Data

In [ ]:
cort_analysis = cort_samples.copy()
cort_analysis.head()

##### Statistics

In [ ]:
steps = [
    ("prep", "normality"),
    ("prep", "equal_var"),
    ("test", "mixed_anova"),
    ("posthoc", "pairwise_ttests")
]
params = {
    "dv": "cortisol",
    "within": "sample",
    "between": "condition",
    "subject": "subject"
}

stats = bp.stats.StatsPipeline(steps, params)
stats.apply(cort_analysis)

stats.export_statistics(stats_path.joinpath("stats_cortisol_interaction.xlsx"))
stats.display_results(prep=False, sig_only="posthoc")

##### Plots

In [ ]:
fig, ax = plt.subplots()
mist.saliva_plot("cortisol", ax=ax, legend_fontsize="medium", hue_order=hue_order)
fig.savefig(plot_path.joinpath("img_corisol_mist.pdf"), transparent=True)

#### Cortisol Features

##### Prepare Data

In [ ]:
features = ["auc_g", "auc_i", "auc_i_post", "max_inc", "slopeS1S4"]

cort_analysis = cort_features.loc[pd.IndexSlice[:, :, features]].copy()
cort_analysis.head()

##### Statistics

In [ ]:
pg.normality()

In [ ]:
steps = [
    ("prep", "normality"),
    ("test", "pairwise_ttests"),
]
params = {
    "dv": "cortisol",
    "between": "condition",
    "groupby": "saliva_feature",
    "test__parametric": False
}

stats = bp.stats.StatsPipeline(steps, params)
stats.apply(cort_analysis)

stats.export_statistics(stats_path.joinpath("stats_cortisol_features.xlsx"))
stats.display_results()

##### Plots

In [ ]:
features = {
    #"auc_g": ["auc_g"],
    "auc": ["auc_i", "auc_i_post"],
    "max_inc": ["max_inc"],
    "slope": ["slopeS1S4"]
}

box_pairs, pvalues = stats.sig_brackets(
    "test", 
    stats_type="between", 
    plot_type="multi", 
    x="saliva_feature", 
    features=features, 
    subplots=True
)

fig, axs = plt.subplots(ncols=len(features.keys()))

bp.protocols.plotting.saliva_multi_feature_boxplot(
    cort_features, 
    "cortisol", 
    features=features, 
    hue="condition", 
    hue_order=hue_order,
    legend_loc="upper center", 
    legend_orientation="horizontal", 
    stats_kwargs={"box_pairs": box_pairs, "pvalues": pvalues},
    axs=axs
); 